# Ion Exchange Simulation - SAC → Na-WAC → Degasser

This notebook performs detailed simulation of the SAC flowsheet for mixed hardness removal.

## Process Description
1. **SAC**: Strong Acid Cation resin removes all hardness (permanent + temporary)
2. **Na-WAC**: Weak Acid Cation resin for alkalinity adjustment and polishing
3. **Degasser**: Strips CO2 to stabilize pH

Best for: Waters with significant permanent hardness (>100 mg/L) or <50% temporary hardness

In [ ]:
# Parameters cell - papermill will inject values here
import json

# Configuration parameters
configuration = None  # Will be injected by papermill
water_analysis = None  # Will be injected by papermill
simulation_options = {
    "model_type": "direct",
    "time_steps": 100,
    "breakthrough_criteria": {"hardness_mg_L_CaCO3": 5.0}
}

In [ ]:
# Import required modules
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Add project root to path
if configuration and 'project_root' in configuration:
    sys.path.insert(0, configuration['project_root'])

from tools.ix_simulation_direct import simulate_ix_system_direct
from tools.schemas import IXSimulationInput, IXConfigurationOutput, MCASWaterComposition

## Na+ Competition Analysis

In [ ]:
# Analyze Na+ competition effects
if water_analysis and configuration:
    na_mg_L = water_analysis['ion_concentrations_mg_L'].get('Na_+', 0)
    ca_mg_L = water_analysis['ion_concentrations_mg_L'].get('Ca_2+', 0)
    mg_mg_L = water_analysis['ion_concentrations_mg_L'].get('Mg_2+', 0)
    
    # Convert to meq/L
    na_meq_L = na_mg_L / 23
    ca_meq_L = ca_mg_L / 20
    mg_meq_L = mg_mg_L / 12.15
    
    total_hardness_meq_L = ca_meq_L + mg_meq_L
    
    print("Na+ Competition Analysis:")
    print(f"Na+ concentration: {na_mg_L:.0f} mg/L ({na_meq_L:.1f} meq/L)")
    print(f"Total hardness: {total_hardness_meq_L:.1f} meq/L")
    print(f"Na/hardness ratio: {na_meq_L/total_hardness_meq_L:.1f}")
    print(f"Competition factor: {configuration.get('na_competition_factor', 1.0):.2f}")
    
    if configuration.get('na_competition_factor', 1.0) < 0.7:
        print("\n⚠️ WARNING: High Na+ competition will reduce SAC capacity significantly!")
        print("Consider more frequent regenerations or higher regenerant levels.")

## Run Simulation

In [ ]:
# Prepare simulation input
if configuration and water_analysis:
    # Create proper schema objects
    config_obj = IXConfigurationOutput(**configuration)
    water_obj = MCASWaterComposition(**water_analysis)
    
    sim_input = IXSimulationInput(
        configuration=config_obj,
        water_analysis=water_obj,
        breakthrough_criteria=simulation_options['breakthrough_criteria'],
        simulation_options=simulation_options
    )
    
    print("Running SAC flowsheet simulation...")
    result = simulate_ix_system_direct(sim_input)
    print(f"Simulation status: {result.status}")

## Breakthrough Curves

In [ ]:
# Plot breakthrough curves for SAC
if result and result.water_quality_progression:
    # Extract SAC breakthrough data
    sac_points = [p for p in result.water_quality_progression if "SAC @" in p.stage]
    
    if sac_points:
        bed_volumes = [float(p.stage.split(" @ ")[1].replace(" BV", "")) for p in sac_points]
        ca_conc = [p.ion_concentrations_mg_L.get('Ca_2+', 0) for p in sac_points]
        mg_conc = [p.ion_concentrations_mg_L.get('Mg_2+', 0) for p in sac_points]
        na_conc = [p.ion_concentrations_mg_L.get('Na_+', 0) for p in sac_points]
        
        plt.figure(figsize=(10, 6))
        plt.plot(bed_volumes, ca_conc, 'b-', label='Ca²⁺', linewidth=2)
        plt.plot(bed_volumes, mg_conc, 'g-', label='Mg²⁺', linewidth=2)
        plt.plot(bed_volumes, na_conc, 'r--', label='Na⁺', linewidth=2)
        
        plt.xlabel('Bed Volumes')
        plt.ylabel('Concentration (mg/L)')
        plt.title('SAC Breakthrough Curves')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Show Na+ release
        feed_na = water_analysis['ion_concentrations_mg_L'].get('Na_+', 0)
        max_na = max(na_conc)
        if max_na > feed_na * 1.1:
            print(f"\n⚠️ Na+ release observed: Peak {max_na:.0f} mg/L vs feed {feed_na:.0f} mg/L")
            print("This is normal as hardness ions displace Na+ from the resin.")

## Performance Comparison

In [ ]:
# Compare vessel performance
if result and result.ix_performance:
    data = []
    for vessel, metrics in result.ix_performance.items():
        data.append({
            'Vessel': vessel,
            'Breakthrough (BV)': metrics.bed_volumes_treated,
            'Runtime (hr)': metrics.breakthrough_time_hours,
            'Regenerant (kg)': metrics.regenerant_consumption_kg,
            'Efficiency (kg/m³)': metrics.regenerant_consumption_kg / metrics.total_throughput_m3,
            'Utilization (%)': metrics.capacity_utilization_percent
        })
    
    df = pd.DataFrame(data)
    print("Vessel Performance Comparison:")
    print(df.to_string(index=False))
    
    # Bar chart comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    vessels = df['Vessel'].tolist()
    x = range(len(vessels))
    
    ax1.bar(x, df['Breakthrough (BV)'], color=['blue', 'green'])
    ax1.set_xticks(x)
    ax1.set_xticklabels(vessels)
    ax1.set_ylabel('Bed Volumes')
    ax1.set_title('Breakthrough Volumes')
    
    ax2.bar(x, df['Regenerant (kg)'], color=['blue', 'green'])
    ax2.set_xticks(x)
    ax2.set_xticklabels(vessels)
    ax2.set_ylabel('Regenerant (kg/cycle)')
    ax2.set_title('Regenerant Consumption')
    
    plt.tight_layout()
    plt.show()

## Operating Cost Analysis

In [ ]:
# Detailed OPEX breakdown
if result and result.economics and 'opex_breakdown' in result.economics:
    opex = result.economics['opex_breakdown']
    
    # Create pie chart
    labels = list(opex.keys())
    values = list(opex.values())
    
    plt.figure(figsize=(8, 8))
    plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=90)
    plt.title('Annual Operating Cost Breakdown')
    plt.axis('equal')
    plt.show()
    
    print("\nAnnual Operating Costs:")
    total_opex = sum(values)
    for label, value in zip(labels, values):
        print(f"  {label}: ${value:,.0f} ({value/total_opex*100:.1f}%)")
    print(f"  TOTAL: ${total_opex:,.0f}")

## SAC-Specific Recommendations

In [ ]:
# Generate SAC-specific operational guidance
if result:
    print("SAC System Operational Guidelines:")
    
    print("\n1. Regeneration Optimization:")
    print("   - Use 10% NaCl brine solution")
    print("   - Regenerant level: 120-160 kg/m³ resin")
    print("   - Consider counter-current regeneration for efficiency")
    
    print("\n2. Monitoring Requirements:")
    print("   - Track hardness breakthrough continuously")
    print("   - Monitor Na+ in product water (will increase)")
    print("   - Check for iron/manganese fouling potential")
    
    print("\n3. Operational Considerations:")
    print("   - SAC handles ALL hardness - no selectivity")
    print("   - Higher salt consumption than WAC systems")
    print("   - Produces acidic water (needs degassing)")
    
    # Na+ competition specific advice
    if configuration and configuration.get('na_competition_factor', 1.0) < 0.7:
        print("\n4. High Na+ Competition Management:")
        print("   - Increase regenerant level to 180-200 kg/m³")
        print("   - Reduce service cycle time by 20-30%")
        print("   - Consider RO pretreatment for Na+ reduction")

In [ ]:
# Prepare results for extraction
results = result.dict() if result else {"status": "error", "message": "No simulation results"}
results